# AWS DevDay Seoul 2019
## 모두를 위한 컴퓨터 비전 딥러닝 툴킷, GluonCV 따라하기
## Lab 2.3 Transfer Learning 적용하기 - YOLOv3

<!-- This notebook is based on: https://github.com/zhreshold/ICCV19-GluonCV -->

노트북을 처음 로딩할 때, Kernel로 **conda_mxnet_p36** 을 선택합니다.

### 랩 순서

3. Object Detection - YOLOv3

이번 랩에서는 앞 세션에 이어서 YOLOv3 object detection 모델에 transfer learning을 적용해 보겠습니다.

#### GluonCV와 필요한 python 패키지를 설치합니다.
GluonCV의 model_zoo와 utils 패키지에 대해서는 아래 링크를 참조하세요.
- model_zoo: [https://gluon-cv.mxnet.io/model_zoo/index.html](https://gluon-cv.mxnet.io/model_zoo/index.html)
- utils: [https://gluon-cv.mxnet.io/api/utils.html](https://gluon-cv.mxnet.io/api/utils.html)

In [ ]:
# 최초 실행시 GPU(p2/p3) instance에서는 아래 코드로 gluoncv 패키지를 설치하세요.
!pip install --upgrade mxnet-cu100mkl gluoncv
# CPU(c4/c5/m4/m5/t2/t3) instance에서는 아래 코드로 gluoncv 패키지를 설치하세요.
#!pip install --upgrade mxnet-mkl gluoncv

# Task 3 - Object Detection - YOLOv3

## 데이터 준비

아래 코드를 실행하면, 실습을 위해 미리 준비된 데이터셋에 접근할 수 있습니다.



In [ ]:
import time
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import numpy as np
import mxnet as mx
from mxnet import autograd, gluon, nd, image
from mxnet.gluon.data.vision import transforms

import gluoncv
from gluoncv.utils import download, viz
from gluoncv.model_zoo import get_model
from gluoncv.utils import viz, download
from gluoncv.data.batchify import Tuple, Stack, Pad
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultTrainTransform
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultValTransform
from gluoncv.data.dataloader import RandomTransformDataLoader
from gluoncv.utils.metrics.voc_detection import VOC07MApMetric, VOCMApMetric
from gluoncv.utils import LRScheduler, LRSequential

#주의: CPU instance를 사용한다면 num_gpus를 0으로 설정하세요.
num_gpus = 1

다운로드한 데이터셋을 로드해서 확인합니다.

In [ ]:
train_dataset = gluoncv.data.RecordFileDetection('train.rec')
val_dataset = gluoncv.data.RecordFileDetection('val.rec')
classes = ['bird', 'boat', 'car', 'person']
image, label = train_dataset[10]
print('label:', label)
# display image and label
ax = viz.plot_bbox(image, bboxes=label[:, :4], labels=label[:, 4:5], class_names=classes)
plt.show()

## Pre-trained 모델

먼저, 여러 object detection 모델 중에서 `YOLO`를 사용해보겠습니다. 빠른 실행을 위해 `darknet53`을 기본으로 하는 `YOLOv3`를 선택합니다.
아래의 코드는 transfer learning을 위해 Pascal VOC로 pretrained 된 `yolo3_darknet53_voc` 모델로부터 커스텀 네트웍을 만듭니다.

`GluonCV Model Zoo`가 지원하는 detection 모델의 전체 목록은 [Model Zoo > Detection](https://gluon-cv.mxnet.io/model_zoo/detection.html) 링크에서 확인할 수 있습니다.

In [ ]:
net = gluoncv.model_zoo.get_model('yolo3_darknet53_custom', classes=classes, transfer='voc')

## Fine-tuning은 학습의 새로운 단계

이제 dataloader를 아래 코드와 같이 정의합니다.




In [ ]:
data_shape = 416
batch_size = 12
num_workers = 2

def get_dataloader(net, train_dataset, val_dataset, data_shape, batch_size, num_workers):
    """Get dataloader."""
    width, height = data_shape, data_shape

    batchify_fn = Tuple(*([Stack() for _ in range(6)] + [Pad(axis=0, pad_val=-1) for _ in range(1)]))  # stack image, all targets generated

    train_loader = gluon.data.DataLoader(
        train_dataset.transform(YOLO3DefaultTrainTransform(width, height, net, mixup=None)),
        batch_size, True, batchify_fn=batchify_fn, last_batch='rollover', num_workers=num_workers)
    
    val_batchify_fn = Tuple(Stack(), Pad(pad_val=-1))
    
    val_loader = gluon.data.DataLoader(
        val_dataset.transform(YOLO3DefaultValTransform(width, height)),
        batch_size, False, batchify_fn=val_batchify_fn, last_batch='keep', num_workers=num_workers)
    
    return train_loader, val_loader

train_data, val_data = get_dataloader(net, train_dataset, val_dataset, data_shape, batch_size, num_workers)



학습에 가능하면 GPU를 사용할 것을 권해드립니다.


In [ ]:
ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
net.collect_params().reset_ctx(ctx)

다음으로 trainer, loss와 metric을 정의합니다.



In [ ]:
trainer = gluon.Trainer(
    net.collect_params(), 'sgd',
    {'learning_rate': 0.001, 'wd': 0.0005, 'momentum': 0.9})

# targets
sigmoid_ce = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=False)
l1_loss = gluon.loss.L1Loss()

# metrics
obj_metrics = mx.metric.Loss('ObjLoss')
center_metrics = mx.metric.Loss('BoxCenterLoss')
scale_metrics = mx.metric.Loss('BoxScaleLoss')
cls_metrics = mx.metric.Loss('ClassLoss')


모든 것이 준비되었으므로 이제 학습을 시작할 수 있습니다. 


In [ ]:
epochs = 30
best_map =[0]

obj_loss_list = []
boxcenter_loss_list = []
boxscale_loss_list = []
classloss_list = []

for epoch in range(0, epochs):
    tic = time.time()
    btic = time.time()
    mx.nd.waitall()
    net.hybridize(static_alloc=True, static_shape=True)
            
    for i, batch in enumerate(train_data):
        batch_size = batch[0].shape[0]
        
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
        # objectness, center_targets, scale_targets, weights, class_targets
        fixed_targets = [gluon.utils.split_and_load(batch[it], ctx_list=ctx, batch_axis=0) for it in range(1, 6)]
        gt_boxes = gluon.utils.split_and_load(batch[6], ctx_list=ctx, batch_axis=0)
        
        sum_losses = []
        obj_losses = []
        center_losses = []
        scale_losses = []
        cls_losses = []
        
        with autograd.record():
            for ix, x in enumerate(data):
                obj_loss, center_loss, scale_loss, cls_loss = net(x, gt_boxes[ix], *[ft[ix] for ft in fixed_targets])
                sum_losses.append(obj_loss + center_loss + scale_loss + cls_loss)
                obj_losses.append(obj_loss)
                center_losses.append(center_loss)
                scale_losses.append(scale_loss)
                cls_losses.append(cls_loss)
            autograd.backward(sum_losses)
        trainer.step(batch_size)
        
        obj_metrics.update(0, obj_losses)
        center_metrics.update(0, center_losses)
        scale_metrics.update(0, scale_losses)
        cls_metrics.update(0, cls_losses)
        
        if i % 100 == 0:
            name1, loss1 = obj_metrics.get()
            name2, loss2 = center_metrics.get()
            name3, loss3 = scale_metrics.get()
            name4, loss4 = cls_metrics.get()                
            print('[Epoch {}][Batch {}], LR: {:.2E}, Speed: {:.3f} samples/sec, {}={:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}'.format(
                epoch, i, trainer.learning_rate, batch_size/(time.time()-btic), name1, loss1, name2, loss2, name3, loss3, name4, loss4))
        btic = time.time()

    name1, loss1 = obj_metrics.get()
    name2, loss2 = center_metrics.get()
    name3, loss3 = scale_metrics.get()
    name4, loss4 = cls_metrics.get()
    
    obj_loss_list.append(loss1)
    boxcenter_loss_list.append(loss2)
    boxscale_loss_list.append(loss3)
    classloss_list.append(loss4)
    
    print('[Epoch {}] Training cost: {:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}'.format(
        epoch, (time.time()-tic), name1, loss1, name2, loss2, name3, loss3, name4, loss4))
    

In [ ]:
plt.plot(obj_loss_list, label='object loss')
plt.plot(classloss_list, label='class loss')
plt.legend()
plt.show()

Fine-tuning의 결과로 얻은 weight 값을 디스크에 저장합니다.

In [ ]:
net.save_parameters('yolo3_darknet53_voc_0000.params')

## Fine-tuning한 모델로 예측하기

위에서 fine-tuning한 weight로 성능을 테스트할 수 있습니다. 


In [ ]:
%%time

net_name = 'yolo3_darknet53_custom'
model_param_fname = 'yolo3_darknet53_voc_0000.params'
# model_param_fname = 'yolo3_darknet53_voc_best.params'
classes = ['bird', 'boat', 'car', 'person']

# ctx = mx.cpu()
ctx = mx.gpu(0)
    
net = gluoncv.model_zoo.get_model(net_name, classes=classes, ctx=ctx, pretrained_base=False)
net.load_parameters(model_param_fname, ctx=ctx)
net.hybridize()

x, image = gluoncv.data.transforms.presets.yolo.load_test('test_sample_0.jpg', 416)
x = x.as_in_context(ctx)
print('Shape of pre-processed image:', x.shape)
cid, score, bbox = net(x)

ax = viz.plot_bbox(image, bbox[0], score[0], cid[0], class_names=classes)
plt.figure(figsize=(32,32))
plt.show()

지금까지 `GluonCV`에서 `YOLOv3` 기반의 커스텀 네트워크를 사용해 object detection하는 예제를 알아봤습니다. 수고하셨습니다.
